<IMG SRC="http://www.lirmm.fr/~poncelet/EGC2021/smallbandeau_EGC2021.png" align="center" >

<H1> Classification de données textuelles </H1>


Lors de l'étape d'ingénierie de données textuelles nous avons vu que diverses opérations pouvaient être appliquées sur les textes et qu'au final il est possible d'obtenir des textes simplifiés. Nous allons, à présent, étudier comment faire de la classification à partir de données textuelles et comment convertir les textes en vecteurs pour pouvoir faire de la classification. 


## **Installation**




Avant de commencer, il est nécessaire de déjà posséder dans son environnement toutes les librairies utiles. Dans la seconde cellule nous importons toutes les librairies qui seront utiles à ce notebook. Il se peut que, lorsque vous lanciez l'éxecution de cette cellule, une soit absente. Dans ce cas il est nécessaire de l'installer. Pour cela dans la cellule suivante utiliser la commande :  

*! pip install nom_librairie*  

**Attention :** il est fortement conseillé lorsque l'une des librairies doit être installer de relancer le kernel de votre notebook.

**Remarque :** même si toutes les librairies sont importées dès le début, les librairies utilisées lors de la présentation d'une fonction dans une cellule sont ré-importées de manière à indiquer d'où elles viennent et ainsi connaîter d'où vient la fonction afin de vous faciliter la réutilisation dans un autre projet.
 

In [ ]:
# utiliser cette cellule pour installer les librairies manquantes
# pour cela il suffit de taper dans cette cellule : !pip install nom_librairie_manquante
# d'exécuter la cellule et de relancer la cellule suivante pour voir si tout se passe bien
# recommencer tant que toutes les librairies ne sont pas installées ...


#!pip install ..

# ne pas oublier de relancer le kernel du notebook

In [ ]:
# Importation des différentes librairies, classes et fonctions utilespour le notebook

#Sickit learn met régulièrement à jour des versions et 
#indique des futurs warnings. 
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


# librairies générales
import pandas as pd
import re
from tabulate import tabulate
import time
import numpy as np
import pickle
import string
import base64

# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns

# librairies scikit learn
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


# librairies des classifiers utilisés
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

# librairies NLTK
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk import word_tokenize 

 
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english')) 

    

Pour pouvoir sauvegarder sur votre répertoire Google Drive, il est nécessaire de fournir une autorisation. Pour cela il suffit d'éxecuter la ligne suivante et de saisir le code donné par Google.

In [ ]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Corriger éventuellement la ligne ci-dessous pour mettre le chemin vers un répertoire spécifique dans votre répertoire Google Drive : 

In [ ]:
%cd /content/gdrive/My Drive/Colab Notebooks/EGC_Ecole2021
#
# pour une utilisation sur une machine locale changer le chemin ci-dessous et décommenter la ligne
#%cd ./

In [3]:
# fonctions utilities (affichage, confusion, etc.)
from MyNLPUtilities import *
import SolClassificationDonneesTextuelles

ModuleNotFoundError: ignored

## **Vectorisation**



L'objectif de la vectorisation est de transformer les documents en vecteurs. Il existe deux approches principales : 
1. L'approche sac de mots dans laquelle il n'y a aucun ordre dans les termes utilisés et qui ne tient compte que du nombre d'occurrences des termes
1. l'approche basée sur TF_IDF qui ne tient pas non plus compte de l'ordre des termes mais qui pondère les valeurs grâce à TF_IDF au lieu de la fréquence des termes.

**Remarque :** même si l'ordre des mots n'est pas pris en compte dans ces approches, les n-grammes peuvent partiellement servir à pallier ce problème. 

### **L'approche Sac de Mots (*Bag of Words*)**




La manière la plus simple de mettre sous la forme de vecteur (*vectorisation*) est d'utiliser les Bag of Words (BOW). Il s'agit, à partir d'une liste de mots (vocabulaire) de compter le nombre d'apparitions du mot du vocabulaire dans le document.  

Cette opération se fait par :
1. Création d'une instance de la classe CountVectorizer.
1. Appel de la fonction fit() pour apprendre le vocabulaire.
1. Appel de la fonction transform() sur un ou plusieurs documents afin de les encoder dans le vecteur.  

La classe CountVectorizer permet également de faire un ensemble de pré-traitement sur un document : mise en minuscule, suppression des stop words (mots vides), suppression des ponctuations ... mais elle ne peut pas lemmatiser ou rechercher les racines des termes. 

Les principaux paramètres utiles sont les suivants :  
1. *lowercase* booléen pour mettre en minuscule le document (défaut=True).
1. *token_pattern* pour éliminer des mots trop petits (défaut=None).
1. *stopwords* pour éliminer les stopwords du document (défaut=None).
1. *analyzer* pour préciser si l'on travaille avec des mots ou des caractères ou appliquer une fonction de pré-traitement (défaut=’word’).
1. *ngram_range* pour pouvoir utiliser des n-grammes de mots ou de caractères en fonction de la valeur d'*analyzer* (défaut=(1, 1), i.e. on ne considère qu'un mot).
1. *max_df* pour ignorer les termes qui ont une fréquence de document strictement supérieure à un seuil donné (termes trop fréquents) (défaut=1.0).
1. *min_df* pour ignorer les termes qui ont une fréquence de document (présence en % de documents) strictement inférieure à un seuil donné (termes peu fréquents) (défaut=1).
1. *max_features* pour limiter le nombre de caractéristiques (*features*) que le vecteur doit contenir (défaut : None).


Pour avoir plus d'information sur CountVectorizer, voir https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html  

Nous décrivons par la suite les différents paramètres.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#premier exemple sans paramètre
texte = ["This is a simple EXAMPLE ! of CountVectorizer for creating a vector"]

print ("document initial ",texte,'\n')
# par defaut conversion en minuscule
vectorizer = CountVectorizer()

# creation du vocabulaire
vectorizer.fit(texte)
# encodage du document
vector = vectorizer.transform(texte)

# la liste des différents features
print ("Les différents features sont",vectorizer.get_feature_names(),' ... à noter tout est converti en minuscule\n')

# Contenu du vocabulaire
print ("Vocabulaire : ")
print(vectorizer.vocabulary_)


# affichage de la taille du vecteur de sortie

print ("\nTaille du vecteur :\n",vector.shape,'\n')

print ("Conversion en mettant lowercase=False")
vectorizer = CountVectorizer(lowercase=False)
# creation du vocabulaire
vectorizer.fit(texte)

# la liste des différents features
print ("Les différents features sont",vectorizer.get_feature_names(),' ... à noter les majuscules sont conservées\n')



document initial  ['This is a simple EXAMPLE ! of CountVectorizer for creating a vector'] 

Les différents features sont ['countvectorizer', 'creating', 'example', 'for', 'is', 'of', 'simple', 'this', 'vector']  ... à noter tout est converti en minuscule

Vocabulaire : 
{'this': 7, 'is': 4, 'simple': 6, 'example': 2, 'of': 5, 'countvectorizer': 0, 'for': 3, 'creating': 1, 'vector': 8}

Taille du vecteur :
 (1, 9) 

Conversion en mettant lowercase=False
Les différents features sont ['CountVectorizer', 'EXAMPLE', 'This', 'creating', 'for', 'is', 'of', 'simple', 'vector']  ... à noter les majuscules sont conservées



Il est possible de combiner *fit* et *transform* comme le montre l'exemple suivant où nous créons également un dataframe pour afficher le vecteur résultat.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

texte = ["This is an example, ! of CountVectorizer for creating a vector",
        "This is another example of CountVectorizer",
        "with or without parameters"]

vectorizer = CountVectorizer()
# fit et transform en une opération
X = vectorizer.fit_transform(texte)

# creation du dataframe pour affichage
df = pd.DataFrame(
    data=vectorizer.transform(texte).toarray(),
    columns=vectorizer.get_feature_names()
)

display(df)

**token_pattern**  

*token_pattern* peut être utilisé pour filtrer uniquement les mots qui font une certaine taille. Elle est, par exemple, fort utile pour supprimer les termes composés d'un seul caractère. Pour cela, il faut préciser une expression régulière.

In [ ]:
texte = ["This is an example, ! of CountVectorizer for creating a vector",
        "This is another example of CountVectorizer",
        "with or without parameters"]

print ("Le vocabulaire ne contient que des mots qui ont plus de trois caractères :")
vectorizer = CountVectorizer(token_pattern=r'\w{3,}')  

# fit et transform en une opération
X = vectorizer.fit_transform(texte)

# creation du dataframe pour affichage
df = pd.DataFrame(
    data=vectorizer.transform(texte).toarray(),
    columns=vectorizer.get_feature_names()
)

display(df)

**stop_words**  

*stop_words* permet de supprimer du vocabulaire les mots qui appartiennent aux stopwords d'une langue (e.g. stopwords='english'). Il se base sur une liste de stopwords définie. Il est également possible de préciser sa propre liste de stopwords. 


In [ ]:
texte = ["This is an example, ! of CountVectorizer for creating a vector",
        "This is another example of CountVectorizer",
        "with or without parameters"]

print ("Le vocabulaire ne contient que des mots qui ne sont pas des stopwords anglais :")
vectorizer = CountVectorizer(stop_words='english')  

# fit et transform en une opération
X = vectorizer.fit_transform(texte)

# creation du dataframe pour affichage
df = pd.DataFrame(
    data=vectorizer.transform(texte).toarray(),
    columns=vectorizer.get_feature_names()
)

display(df)

print ("Le vocabulaire ne contient que des mots qui ne sont pas dans une liste spécifiée de stopwords (example, vector, creating) : ")
vectorizer = CountVectorizer(stop_words=['example','vector','creating'])  

# fit et transform en une opération
X = vectorizer.fit_transform(texte)

# creation du dataframe pour affichage
df = pd.DataFrame(
    data=vectorizer.transform(texte).toarray(),
    columns=vectorizer.get_feature_names()
)

display(df)

**Les n-grammes**  
Il est possible de préciser que les features sont composés de n-grammes à l'aide du paramètre *ngram_range*. Ce dernier spécifie l'intervalle de valeurs possibles. Par exemple *ngram_range=(1, 2)* permettra d'obtenir des n-grammes de taille 1 et 2 mots, *ngram_range=(1, 3)* des n-grammes de 1, 2 et 3 mots, *ngram_range=(3, 3)* des n-grammes de 3 mots, etc.  

Par défaut, il s'agit de n-grammes de mots, pour avoir des n-grammes de caractères, le paramètre *analyzer* doit être initialisé avec *analyzer='char'*. 

In [ ]:
texte = ["This is an example of CountVectorizer for creating a vector",
        "This is another example of CountVectorizer",
        "with or without parameters"]

print ("n-grammes de mots de taille 1")
vectorizer = CountVectorizer(ngram_range=(1,1))
# fit et transform en une opération
X = vectorizer.fit_transform(texte)
print (vectorizer.get_feature_names())

print ("\nn-grammes de mots de taille 1 et 2")
vectorizer = CountVectorizer(ngram_range=(1,2))
# fit et transform en une opération
X = vectorizer.fit_transform(texte)
print (vectorizer.get_feature_names())

print ("\nn-grammes de mots de taille 2 et 3")
vectorizer = CountVectorizer(ngram_range=(2,3))
# fit et transform en une opération
X = vectorizer.fit_transform(texte)
print (vectorizer.get_feature_names())

print ("\nn-grammes de mots de taille 3")
vectorizer = CountVectorizer(ngram_range=(3,3))
# fit et transform en une opération
X = vectorizer.fit_transform(texte)
print (vectorizer.get_feature_names())


print ("\nn-grammes de caractères de taille 1 et 2")
vectorizer = CountVectorizer(analyzer='char',ngram_range=(1,2))
# fit et transform en une opération
X = vectorizer.fit_transform(texte)
print (vectorizer.get_feature_names())

**min_df et max_df**  

*min_df* ignore les termes qui ont une fréquence de document (présence en % de documents) strictement inférieure au seuil donné. Par exemple, *min_df = 0,55* exige qu'un terme apparaisse dans 55% des documents pour être considéré comme faisant partie du vocabulaire. *max_df* à l'inverse ignore les termes qui sont supérieurs au seuil. Il est utilisé, par exemple, pour éliminer les termes trop fréquents.

In [ ]:
texte = ["This is an example of CountVectorizer for creating a vector",
        "This is another example of CountVectorizer",
        "with or without parameters"]


print ("Ne conserver que les termes qui apparaissent dans au moins 50% des documents avec min_df=0.5")
vectorizer = CountVectorizer(min_df=0.5)
# fit et transform en une opération
X = vectorizer.fit_transform(texte)
print (vectorizer.get_feature_names())

print ("Ne conserver que les termes qui sont inférieurs à 50% des documents avec max_df=0.5")
vectorizer = CountVectorizer(max_df=0.5)
# fit et transform en une opération
X = vectorizer.fit_transform(texte)
print (vectorizer.get_feature_names())

Ne conserver que les termes qui apparaissent dans au moins 50% des documents avec min_df=0.5
['countvectorizer', 'example', 'is', 'of', 'this']
Ne conserver que les termes qui sont inférieurs à 50% des documents avec max_df=0.5
['an', 'another', 'creating', 'for', 'or', 'parameters', 'vector', 'with', 'without']


**max_features**  
*max_features* permet de préciser la taille de sortie du vecteur, i.e. le nombre de termes à conserver.

In [ ]:
texte = ["This is an example of CountVectorizer for creating a vector",
        "This is another example of CountVectorizer",
       "with or without parameters"]

print ("Ne conserver que 8 features pour le vocabulaire")
vectorizer = CountVectorizer(max_features=8)
# fit et transform en une opération
X = vectorizer.fit_transform(texte)
print (vectorizer.get_feature_names())

print ("Pas de contraintes sur la taille du vocabulaire")
vectorizer = CountVectorizer()
# fit et transform en une opération
X = vectorizer.fit_transform(texte)
print (vectorizer.get_feature_names())



Ne conserver que 8 features pour le vocabulaire
['an', 'another', 'countvectorizer', 'creating', 'example', 'is', 'of', 'this']
Pas de contraintes sur la taille du vocabulaire
['an', 'another', 'countvectorizer', 'creating', 'example', 'for', 'is', 'of', 'or', 'parameters', 'this', 'vector', 'with', 'without']


**Remarque :** l'inconvénient de CountVectorizer est qu'il génère des matrices qui sont creuses, i.e. il y a beaucoup de zéros. L'exemple suivant illustre le contenu de la matrice précédente où le bleu foncé indique qu'il y a une valeur et le bleu claire indique un zéro.

In [ ]:
sns.heatmap(X.todense(), cmap="Blues", vmin=0, vmax=1).set_title('Matrice Creuse pour CountVectorizer')

### **L'approche via TF_IDF** 



Le but de l'utilisation de tf-idf est de réduire l'impact des termes qui apparaissent très fréquemment dans un corpus donné et qui sont donc moins informatifs que les autres termes dans le corpus d'apprentissage. 

CountVectorizer, en prenant en compte l'occurrence des mots, est souvent trop limité. Une alternative est d'utiliser la 
mesure TF-IDF (Term Frequency – Inverse Document) qui a pour but de réduire l'impact des termes qui apparaissent très fréquemment dans un corpus donné :  

$
 tf$-$idf(d, t) = tf(t) * idf(d, t)
$  

où $tf(t)$= la fréquence du terme, i.e. le nombre de fois où le terme apparaît dans le document  
et $idf(d, t)$ = la fréquence du document, i.e. le nombre de documents 'd' qui contiennent le terme 't'. 

Le principe est le même que pour CountVectorizer, cette opération se fait par :
1. Création d'une instance de la classe TfidfVectorizer.
1. Appel de la fonction fit() pour apprendre le vocabulaire.
1. Appel de la fonction transform() sur un ou plusieurs documents afin de les encoder dans le vecteur.  

Les paramètres sont assez similaires à ceux de CountVectorizer, voir https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html


**Remarque :** Il est possible, si CountVectorizer a déjà été utilisé, de le faire suivre par TfidfTransformer pour simplement mettre à jour les valeurs.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

texte = ["This is an example of TfidfVectorizer for creating a vector",
        "This is another example of TfidfVectorizer",
        "with or without parameters"]

print ("Application de TfidfVectorizer :")
vectorizer = TfidfVectorizer()
# fit et transform en une opération
X = vectorizer.fit_transform(texte)

# creation du dataframe pour affichage
df = pd.DataFrame(
    data=vectorizer.transform(texte).toarray(),
    columns=vectorizer.get_feature_names()
)

display(df)

Il est possible d'obtenir l'idf de chaque terme du vocabulaire via l'attribut *idf_* :

In [ ]:
print ("Affichage de l'idf de chaque terme du vocabulaire : ");

print(dict(zip(vectorizer.get_feature_names(), vectorizer.idf_)))

Un exemple combinant différents attributs :

In [ ]:
texte = ["This is an example of TfidfVectorizer for creating a vector",
        "This is another example of TfidfVectorizer",
        "with or without parameters"]

print ("Application de TfidfVectorizer avec ngram_range=(1,2) et suppression des stopwords :")
vectorizer = TfidfVectorizer(stop_words='english',ngram_range=(1,2))
# fit et transform en une opération
X = vectorizer.fit_transform(texte)

# creation du dataframe pour affichage
df = pd.DataFrame(
    data=vectorizer.transform(texte).toarray(),
    columns=vectorizer.get_feature_names()
)

display(df)

**Remarque :** l'un des gros avantages de TfidfVectorizer par rapport à CountVectorizer est qu'il génère des matrices moins creuses.

<font color=red>Exercice :</font> CountVectorizer et TfidfVectorizer ne possèdent pas de dictionnaire de stop words français. Cependant nous avons vu qu'il était possible de passer comme paramètre une liste de stopwords à supprimer.  
Télécharger le fichier : StopWordsFrench.csv et sauvegarder le sur votre répertoire courant.  

Pour cela vous pouvez utiliser directement la commande :  




In [ ]:
!wget https://www.lirmm.fr/~poncelet/EGC2021/StopWordsFrench.csv  

Cette liste a été obtenue à partir du site : https://referencement-gratuit.and-co.ch/download/liste-stop-words-francais.txt

Compléter la cellule suivante de manière à créer un pipeline qui élimine les stopwords français et détermine des n-grammes d'intervalle 1 à 2, que les features ne soient pas convertis en minuscule et qu'au final le nombre de features soit égal à 15. Il faut tester à la fois pour CountVectorizer et TfidfVectorizer. 

In [ ]:
list_french_stopwords=pd.read_csv("StopWordsFrench.csv", sep=',',index_col = 0)
# conversion en liste
list_french_stopwords=list_french_stopwords.values.tolist()


texte = ["Au clair de la lune",
         "mon ami Pierrot",
        "Prête-moi ta plume",
        "Pour écrire un mot",
        "Ma chandelle est morte"
        "Je n'ai plus de feu"]


<font color=blue>Solution :</font>

In [ ]:
# Pour CountVectorizer
vectorizer = CountVectorizer(lowercase=False,stop_words=list_french_stopwords,ngram_range=(1,2),max_features=15)
# fit et transform en une operation
X = vectorizer.fit_transform(texte)

# creation du dataframe pour affichage
df = pd.DataFrame(
    data=vectorizer.transform(texte).toarray(),
    columns=vectorizer.get_feature_names()
)

display(df)
sns.heatmap(X.todense(), cmap='Blues', vmin=0, vmax=1).set_title('Matrice pour CountVectorizer')

# Pour TF-IDF
vectorizer = TfidfVectorizer(lowercase=False,stop_words=list_french_stopwords,ngram_range=(1,2),max_features=15)
X = vectorizer.fit_transform(texte)


df = pd.DataFrame(
    data=vectorizer.transform(texte).toarray(),
    columns=vectorizer.get_feature_names()
)

display(df)

## **Prise en compte des prétraitements avant transformation**  



Précédemment nous avons vu qu'il était possible d'appliquer de très nombreux pré-traitements sur les documents. Même si CountVectorizer et TfidfVectorizer offrent certaines fonctionnalités, ces dernières ne sont pas forcément suffisantes. Dans cette section, nous présentons comment les pipelines peuvent être utilisés pour mettre en place une chaîne de traitement qui pré-traite les données pour les convertir en vecteurs.  

Considérons la fonction suivante qui effectue un certain nombre de pré-traitements sur un seul document. Par défaut, les paramètres sont à False pour effectuer les pré-traitements. Pour l'activer un pré-traitement, il suffit de mettre le paramètre à True. 


In [ ]:
import re
import string

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk import word_tokenize

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english')) 

def MyCleanText(X, 
               lowercase=False, # mettre en minuscule
               removestopwords=False, # supprimer les stopwords
               removedigit=False, # supprimer les nombres  
               getstemmer=False, # conserver la racine des termes
               getlemmatisation=False # lematisation des termes 
              ):
    
    sentence=str(X)

    # suppression des caractères spéciaux
    sentence = re.sub(r'[^\w\s]',' ', sentence)
    # suppression de tous les caractères uniques
    sentence = re.sub(r'\s+[a-zA-Z]\s+', ' ', sentence)
    # substitution des espaces multiples par un seul espace
    sentence = re.sub(r'\s+', ' ', sentence, flags=re.I)

    # decoupage en mots
    tokens = word_tokenize(sentence)
    if lowercase:
          tokens = [token.lower() for token in tokens]

    # suppression ponctuation
    table = str.maketrans('', '', string.punctuation)
    words = [token.translate(table) for token in tokens]

    # suppression des tokens non alphabetique ou numerique
    words = [word for word in words if word.isalnum()]
    
    # suppression des tokens numerique
    if removedigit:
        words = [word for word in words if not word.isdigit()]

    # suppression des stopwords
    if removestopwords:
        words = [word for word in words if not word in stop_words]

    # lemmatisation
    if getlemmatisation:
        lemmatizer=WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word)for word in words]
        

    # racinisation
    if getstemmer:
        ps = PorterStemmer()
        words=[ps.stem(word) for word in words]
        
    sentence= ' '.join(words)
  
    return sentence   

L'exemple suivant illustre 3 cas d'utilisation : 

In [ ]:
texte = """This is an example of using the Function MyCleanText before creating a vector created, \
          this text has some problems like 1 c or even numbers like 13 and we have corpora"""

print ("Texte d'origine :\n", texte,'\n')
print ('Utilisation de MyCleanText avec les paramètres par défaut (nettoyage des caractères spéciaux, des caractères uniques etc)')
print (MyCleanText(texte),'\n')

print ('Utilisation de MyCleanText avec convertion en minuscule, en prenant les racines, en supprimant les nombres')
print (MyCleanText(texte,lowercase=True,getstemmer=True, removedigit=True),'\n')

print ('Utilisation de MyCleanText avec convertion en minuscule et en mettant sous la forme de lemmes')
print (MyCleanText(texte,lowercase=True,getlemmatisation=True),'\n')

**Les estimateurs et Transformer**  

Scikit learn propose une interface Transformer qui est un type spécial d'estimateur qui crée un nouvel ensemble de données à partir d'un ancien en fonction de règles apprises lors de l'appel à la fonction *fit*. Il existe de très nombreux Transformer dans Scikit-Learn pour normaliser, mettre à l'échelle, gérer les valeurs manquantes, réduire les dimensions, etc. 
De nombreuses informations sur les estimateurs proposés et leurs utilisations sont disponibles ici : https://scikit-learn.org/stable/developers/develop.html  

L'interface de base pour un Transformer est la suivante : 

from sklearn.base import TransformerMixin

class Transfomer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        """
        Apprendre comme transformer les données en fonction des données d'entrées X.
        """
        return self

    def transform(self, X):
        """
        Transformer X dans un nouveau jeu de données Xprime et le retourner.
        """
        return Xprime

où via la méthode Transformer.transform nous pouvons transformer les données initiales.    

Nous pouvons par exemple construire la classe *TextNormalizer* qui effectue les pré-traitements sur les données (suppression stopwords, récupération des racines, etc.) définis dans la fonction MyCleanText 

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self, 
                 removestopwords=False, # suppression des stopwords
                 lowercase=False,# passage en minuscule
                 removedigit=False, # supprimer les nombres  
                 getstemmer=False,# racinisation des termes 
                 getlemmatisation=False # lemmatisation des termes  
                ):
        
        self.lowercase=lowercase
        self.getstemmer=getstemmer
        self.removestopwords=removestopwords
        self.getlemmatisation=getlemmatisation
        self.removedigit=removedigit

    def transform(self, X, **transform_params):
        # Nettoyage du texte
        X=X.copy() # pour conserver le fichier d'origine
        return [MyCleanText(text,lowercase=self.lowercase,
                            getstemmer=self.getstemmer,
                            removestopwords=self.removestopwords,
                            getlemmatisation=self.getlemmatisation,
                            removedigit=self.removedigit) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self
    
    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X).transform(X)

    def get_params(self, deep=True):
        return {
            'lowercase':self.lowercase,
            'getstemmer':self.getstemmer,
            'removestopwords':self.removestopwords,
            'getlemmatisation':self.getlemmatisation,
            'removedigit':self.removedigit
        }    
    
    def set_params (self, **parameters):
        for parameter, value in parameters.items():
            setattr(self,parameter,value)
        return self    


Il est donc maintenant possible d'enchaîner des pré-traitement et l'application d'un tf-idf.

In [ ]:
texte = ["This is an example of TfidfVectorizer for creating a vector",
        "This is another example of TfidfVectorizer",
        "but before we apply a preprocessing"]

print ("texte avant ",texte)
# il suffit de créer un objet de la classe TextNormalizer
text_normalizer=TextNormalizer(lowercase=True)  
# d'appliquer fit.transform pour appliquer les pré-traitements
text_cleaned=text_normalizer.fit_transform(texte)
print ("texte après application des pré-traitements")
print (text_cleaned)     

# pour l'enchainer avec un tf-idf : 
tfidf=TfidfVectorizer(ngram_range=(2, 2))
vector_tfidf=tfidf.fit_transform(text_cleaned)
print ("texte transformé en vecteur tf-idf")
print (vector_tfidf)

# le vecteur peut par la suite être transformé en matrice : 
print ("transformation du vecteur en matrice")
vector_tfidf.toarray()

# notons que cette matrice pourra être à l'entrée d'un classifier

La généralisation du principe précédent via un pipeline se fait alors simplement : 

In [ ]:
from sklearn.pipeline import Pipeline

texte = ["This is an example of TextNormalizer then TfidfVectorizer for creating a vector",
        "This is not another example of CountVectorizer",
        "with or without parameters. Rather is a mainly a pipeline with more or less default parameters"]

pipe = Pipeline([("cleaner", TextNormalizer()),
                 ("count_vectorizer", TfidfVectorizer(ngram_range=(2, 2)))])
pipe.fit(texte)
pipe.transform(texte)


# creation du dataframe pour affichage
# il est possible d'accèder à une étape du pipeline en spécifiant le nom (e.g. pipe['cleaner'])
# dans le dataframe on récupère les différents features comme nom de colonnes
df = pd.DataFrame(
    data=pipe.transform(texte).toarray(),
    columns=pipe['count_vectorizer'].get_feature_names()
)

display(df)

**Attention :** pour rappel CountVectorizer ou TfidfVectorizer mettent en minuscule par défaut. 


In [ ]:
texte = ["This is an example of CountVectorizer for creating a vector",
        "This is another example of CountVectorizer",
        "with or without parameters"]

pipe = Pipeline([("cleaner", TextNormalizer(removestopwords=False,lowercase=False)),
                 ("count_vectorizer", TfidfVectorizer(lowercase=False))])
pipe.fit(texte)
pipe.transform(texte)


df = pd.DataFrame(
    data=pipe.transform(texte).toarray(),
    columns=pipe['count_vectorizer'].get_feature_names()
)

display(df)

## **Exemple de classification**

Maintenant que nous savons pré-traiter les données et construire des vecteurs nous pouvons passer à l'étape de classification. 
Le jeu de donnée que nous allons utiliser est tiré de la base de l'UCI : https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences
et a été créée par : "From Group to Individual Labels using Deep Features', Kotzias et. al,. KDD 2015".  

Il contient des phrases d'avis de trois sites différents (Yeld, Amazon et Imbd) et contient pour chacun de ces sites 500 avis positifs (valeur = 1) et 500 avis négatif (valeur = 0).
Le site : https://archive.ics.uci.edu/ml/machine-learning-databases/00331/
propose 3 fichiers de textes bruts nommés amazon_cells_labelled.txt, imdb_labelled.txt et yelp_labelled.txt. 

Il est possible de télécharger une version regroupant les trois fichiers en un seul fichier ici : https://www.lirmm.fr/~poncelet/ReviewsLabelled.csv  

Pour le télécharger depuis le notebook vous pouvez exécuter la cellule suivante : 




In [ ]:
!wget https://www.lirmm.fr/~poncelet/EGC2021/ReviewsLabelled.csv

Lecture du fichier 

In [ ]:
df = pd.read_csv("ReviewsLabelled.csv", names=['sentence','sentiment','source'], header=0,sep='\t', encoding='utf8')

print ("les 10 premières lignes du fichier :")
display(df[0:10])
print ("la taille du fichier : ", df.shape)
print ("le nombre d'avis différents : \n",df['sentiment'].value_counts(),'\n')
print ("Un exemple d'avis \n",df['sentence'][0],'\n')

# selection des données
X=df.sentence
y=df.sentiment

## **Quel est le meilleur pré-traitement et la meilleure représentation de vecteur ?**

Précédemment nous avons vu comment pré-traiter les données et comment transformer les données en vecteur. Par exemple, appliquons le pipeline sur les différentes phrases du corpus de données pour les transformer : 


In [ ]:
# création du pipeline
pipe = Pipeline([("cleaner", TextNormalizer()),
                 ("count_vectorizer", TfidfVectorizer())])
pipe.fit(X)
pipe.transform(X)

# creation du dataframe pour affichage
df_pipe = pd.DataFrame(
    data=pipe.transform(X).toarray(),
    columns=pipe['count_vectorizer'].get_feature_names()
)

display(df_pipe)

**Un premier essai simple de classification**  

L'objectif ici est d'utiliser un premier classifier et d'étudier quels sont les pré-traitements et vecteurs les plus efficaces pour ce dernier. Pour commencer nous prendrons SVM qui obtient souvent de bons résultats sur les données textuelles. 

Nous pouvons donc, pour simplifier, créer un jeu d'apprentissage et un jeu de test et évaluer le résultat d'un classifieur SVM placé dans un pipeline.


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split


# Création d'un jeu d'apprentissage et de test
trainsize=0.7 # 70% pour le jeu d'apprentissage, il reste 30% du jeu de données pour le test

testsize= 0.3
seed=30
X_train,X_test,y_train,y_test=train_test_split(X, 
                                               y, 
                                               train_size=trainsize, 
                                               random_state=seed,
                                               test_size=testsize)

# création du pipeline en ajoutant le classifier
pipe = Pipeline([("cleaner", TextNormalizer()),
                 ("count_vectorizer", CountVectorizer()),
                 ("SVM", SVC())])
pipe.fit(X_train,y_train)

print("pipeline créé")

pipeline créé


Prediction pour évaluer la qualité du modèle appris : 

In [ ]:
#from sklearn import metrics


y_pred = pipe.predict(X_test)

MyshowAllScores(y_test,y_pred)


L'objectif à présent est de pouvoir tester plusieurs pré-traitement et vectorisation afin de déterminer ceux qui amènent à la meilleure classification. Pour cela il suffit de créer autant de pipeline que l'on souhaite tester.

In [ ]:
# pipeline de l'utilisation de CountVectorizer sur le texte presque sans traitement
CV_brut = Pipeline([("cleaner", TextNormalizer(removestopwords=True,lowercase=True,
                                               getstemmer=True,removedigit=True)), 
                    ("count_vectorizer", CountVectorizer(lowercase=False)),
                    ("SVM", SVC())])

# pipeline de l'utilisation de TfidfVectorizer sur le texte presque sans traitement
TFIDF_brut = Pipeline ([("cleaner", TextNormalizer(removestopwords=True,lowercase=True,
                                               getstemmer=True,removedigit=True)), 
                    ("count_vectorizer", TfidfVectorizer(lowercase=False)),
                    ("SVM", SVC())])

# Liste de tous les modèles à tester
all_models = [
    ("CV_brut", CV_brut),
    ("TFIDF_brut", TFIDF_brut)
]

L'évaluation se fait en utilisant cross_val_score qui effectue une cross validation et évalue le modèle.

In [ ]:
from sklearn.model_selection import cross_val_score


# Evaluation des différents pipelines
unsorted_scores = [(name, cross_val_score(model, X, y, cv=5).mean()) for name, model in all_models]
scores = sorted(unsorted_scores, key=lambda x: -x[1])


print (tabulate(scores, floatfmt=".3f", headers=("Pipeline", 'Score')))


<font color=red>Exercice :</font> En vous inspirant du code précédent, proposer la combinaison pré-traitement, vectorisation qui permette d'obtenir le meilleur score de classification. 

<font color=blue> Solution :</font>
  

In [ ]:
# le plus simple est de faire un test sur differents pipelines.  
# pipeline de l'utilisation de CountVectorizer sur le texte avec differents pre-traitements
CV_brut = Pipeline([('cleaner', TextNormalizer()), 
                    ('count_vectorizer', CountVectorizer(lowercase=False)),
                    ('svm', SVC())])
CV_lowcase = Pipeline([('cleaner', TextNormalizer(removestopwords=False,lowercase=True,
                                               getstemmer=False,removedigit=False)), 
                    ('count_vectorizer', CountVectorizer(lowercase=False)),
                    ('svm', SVC())])
CV_lowStop = Pipeline([('cleaner', TextNormalizer(removestopwords=True,lowercase=True,
                                               getstemmer=False,removedigit=False)), 
                    ('count_vectorizer', CountVectorizer(lowercase=False)),
                    ('svm', SVC())])

CV_lowStopstem = Pipeline([('cleaner', TextNormalizer(removestopwords=True,lowercase=True,
                                               getstemmer=True,removedigit=False)), 
                    ('count_vectorizer', CountVectorizer(lowercase=False)),
                    ('svm', SVC())])

# pipeline de l'utilisation de TfidfVectorizer avec differents pre-traitements
TFIDF_brut = Pipeline ([('cleaner', TextNormalizer()), 
                    ('tfidf_vectorizer', TfidfVectorizer(lowercase=False)),
                    ('svm', SVC())])

TFIDF_lowcase = Pipeline([('cleaner', TextNormalizer(removestopwords=False,lowercase=True,
                                               getstemmer=False,removedigit=False)), 
                    ('tfidf_vectorizer', TfidfVectorizer(lowercase=False)),
                    ('svm', SVC())])
TFIDF_lowStop = Pipeline([('cleaner', TextNormalizer(removestopwords=True,lowercase=True,
                                               getstemmer=False,removedigit=False)), 
                    ('tfidf_vectorizer', TfidfVectorizer(lowercase=False)),
                    ('svm', SVC())])

TFIDF_lowStopstem = Pipeline([('cleaner', TextNormalizer(removestopwords=True,lowercase=True,
                                               getstemmer=True,removedigit=False)), 
                    ('tfidf_vectorizer', TfidfVectorizer(lowercase=False)),
                    ("svm", SVC())])


# Liste de tous les modeles à tester
all_models = [
    ("CV_brut", CV_brut),
    ("CV_lowcase", CV_lowcase),
    ("CV_lowStop", CV_lowStop),
    ("CV_lowStopstem",CV_lowStopstem),
    ("TFIDF_lowcase", TFIDF_lowcase),
    ("TFIDF_lowStop", TFIDF_lowStop),
    ("TFIDF_lowStopstem",TFIDF_lowStopstem),
    ("TFIDF_brut", TFIDF_brut)
]


# Evaluation des differents pipelines
print ("Evaluation des différentes configurations : ")
unsorted_scores = [(name, cross_val_score(model, X, y, cv=5).mean()) for name, model in all_models]
scores = sorted(unsorted_scores, key=lambda x: -x[1])


print (tabulate(scores, floatfmt='.4f', headers=('Pipeline', 'Score')))

## **Evaluation de différents classifieurs**  



Dans cette section, nous évaluons différents classifieurs pour voir lequel est le plus performant. Comme nous appliquons pour chaque classifier les mêmes pré-traitements (appel de TextNormalizer sans paramètres) et l'obtention de la matrice pour tf-idf. Plutôt que de faire des pipelines et de relancer cette étape pour chaque classifier nous la réalisons en premier. Puis nous testons les différents classifiers via une cross validation.  

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

# creation du tableau des différents classifieur 


models = []
models.append(('MultinomialNB',MultinomialNB()))
models.append(('LR', LogisticRegression(solver='lbfgs')))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('SVM', SVC()))



In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


score = 'accuracy'
seed = 7        
allresults = []
results = []
names = []

X=df.sentence
y=df.sentiment

# Nous appliquons les pré-traitements sur X

text_normalizer=TextNormalizer()  
# appliquer fit.transform pour réaliser les pré-traitements sur X
X_cleaned=text_normalizer.fit_transform(X)

# pour l'enchainer avec un tf-idf et obtenir une matrice
tfidf=TfidfVectorizer()
features=tfidf.fit_transform(X_cleaned).toarray()

# attention ici il faut passer features dans cross_val_score plutôt que X
    
for name,model in models:
    # cross validation en 10 fois
    kfold = KFold(n_splits=10, random_state=seed)
    
    print ("Evaluation de ",name)
    start_time = time.time()
    # application de la classification
    cv_results = cross_val_score(model, features, y, cv=kfold, scoring=score)
    
    # pour afficher les paramètres du modèle en cours et la taille du vecteur intermédiaire
    # enlever le commentaire des deux lignes suivantes 
    #print ("paramètre du modèle ",model.get_params(),'\n')
    #print ("taille du vecteur : ",(model.named_steps['tfidf_vectorizer'].fit_transform(X)).shape,'\n')

    thetime=time.time() - start_time
    result=Result(name,cv_results.mean(),cv_results.std(),thetime)
    allresults.append(result)
    # pour affichage
    results.append(cv_results)
    names.append(name)
    print("%s : %0.3f (%0.3f) in %0.3f s" % (name, cv_results.mean(), cv_results.std(),thetime))         
    
allresults=sorted(allresults, key=lambda result: result.scoremean, reverse=True) 

# affichage des résultats
print ('\nLe meilleur resultat : ')
print ('Classifier : ',allresults[0].name, 
       ' %s : %0.3f' %(score,allresults[0].scoremean), 
       ' (%0.3f)'%allresults[0].stdresult,  
       ' en %0.3f '%allresults[0].timespent,' s\n')

print ('Tous les résultats : \n')
for result in allresults:
    print ('Classifier : ',result.name, 
       ' %s : %0.3f' %(score,result.scoremean), 
       ' (%0.3f)'%result.stdresult,  
       ' en %0.3f '%result.timespent,' s')

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
fig.suptitle('Comparaison des algorithmes')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)

<font color=red>Exercice :</font> En vous inspirant du code précédent, évaluer les différents classifiers non plus par rapport à l'accuracy mais par rapport au rappel (*recall*) ou à la précision (*precision*). 

<font color=blue>Solution :</font> 

In [ ]:
# il suffit de remplacer dans le code score ='recall' ou score='precision'.
print ("Pour le rappel : ")
score = 'recall'       
allresults = []
    
for name,model in models:
    kfold = KFold(n_splits=10, random_state=seed)
    print ("Evaluate ",name, 'pour ',score)
    start_time = time.time()
    # application de la classification
    cv_results = cross_val_score(model, features, y, cv=kfold, scoring=score)
    
    thetime=time.time() - start_time
    result=Result(name,cv_results.mean(),cv_results.std(),thetime)
    allresults.append(result)
    print("%s : %0.3f (%0.3f) in %0.3f s" % (name, cv_results.mean(), cv_results.std(),thetime))         
    
allresults=sorted(allresults, key=lambda result: result.scoremean, reverse=True) 

print ('Le meilleur resultat : ')
print ('Classifier : ',allresults[0].name, 
       ' %s : %0.3f' %(score,allresults[0].scoremean), 
       ' (%0.3f)'%allresults[0].stdresult,  
       ' en %0.3f '%allresults[0].timespent,' s')

print ()
print ('Tous les résultats : ')
for result in allresults:
    print ('Classifier : ',result.name, 
       ' %s : %0.3f' %(score,result.scoremean), 
       ' (%0.3f)'%result.stdresult,  
       ' en %0.3f '%result.timespent,' s')
    
    
print ()    
print ("Pour la precision : ")
score = 'precision'       
allresults = []
    
for name,model in models:
    kfold = KFold(n_splits=10, random_state=seed)
    print ("Evaluate ",name, 'pour ',score)
    start_time = time.time()
    # application de la classification
    cv_results = cross_val_score(model, features, y, cv=kfold, scoring=score)
    
    thetime=time.time() - start_time
    result=Result(name,cv_results.mean(),cv_results.std(),thetime)
    allresults.append(result)
    print("%s : %0.3f (%0.3f) in %0.3f s" % (name, cv_results.mean(), cv_results.std(),thetime))         
    
allresults=sorted(allresults, key=lambda result: result.scoremean, reverse=True) 

print ('Le meilleur resultat : ')
print ('Classifier : ',allresults[0].name, 
       ' %s : %0.3f' %(score,allresults[0].scoremean), 
       ' (%0.3f)'%allresults[0].stdresult,  
       ' en %0.3f '%allresults[0].timespent,' s')

print ('Tous les résultats : ')
for result in allresults:
    print ('Classifier : ',result.name, 
       ' %s : %0.3f' %(score,result.scoremean), 
       ' (%0.3f)'%result.stdresult,  
       ' en %0.3f '%result.timespent,' s') 

**Remarque :** dans les code précédents les différentes opérations efffectuées se font pour chaque classifieur : l'application des pré-traitements et la vectorisation avec TfidfVectorizer. Il aurait été bien sûr possible d'effectuer les prétraitements et vectorisation au préalable et d'appliquer uniquement les classifiers sur les données transformées.



## **Recherche des hyperparamètres**



Nous avons vu que SVM obtenait de bons résultats par rapport aux autres. Dans cette section nous étudions les hyperparamètres. Précédemment, nous avons vu après les avoir testé quels étaient également les meilleurs paramètres pour les pré-traitement et la vectorisation. Nous montrons aussi qu'il est possible de rechercher en même temps et de manière automatique quels sont les meilleurs paramètres.

SVM dispose de différents hyperparamètres qui peuvent être pris en compte dont les principaux sont :  
1. Le Noyaux (*kernel*) dont La fonction principale est de prendre un espace d'entrée de faible dimension et de le transformer en un espace de dimension supérieure. Il est surtout utile dans les problèmes de séparation non linéaire. Scikit learn propose les noyaux suivants : 'linear','poly','rbf','sigmoid'.
1. *C* (*regularization*) qui est le paramètre de pénalité, qui représente une mauvaise classification ou un terme d'erreur. Le terme de classification erronée ou d'erreur indique à l'optimisation SVM le niveau d'erreur supportable. Il permet de contrôler le compromis entre la frontière de décision un élément mal classé. En général plus C est grand mieux il classera les données mais cela entraîne aussi des fois du supapprentissage (*overfitting*). Inversement un C trop petit peut entraîner du sous-apprentissage (*underfitting*).
1. *Gamma* qui définit jusqu'où l'influence d'un seul exemple d'entraînement peut aller, avec des valeurs faibles signifiant «loin» et des valeurs élevées signifiant «proche». Lorsque Gamma est élevé, les points proches auront une forte influence; un gamma faible signifie par contre que des points éloignés doivent également être pris en compte pour obtenir la limite de décision.  

Vous pourrez trouver plus d'informations sur les hyperparamètres de SVM sous scikit learn ici : https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html  

Nous pouvons donc créer, à présent, un pipeline et spécifier les hyperparamètres à tester via GridSearchCV. Dans un premier temps, nous considérons un pipeline composé uniquent de la vectorisation et du classifier.  

**Remarque :** GridSearchCV effectue par défaut une cross validation (*cv*) avec une valeur par défaut de 5. Par contre étant donné qu'il fonctionne sur l'ensemble des données, il n'est pas possible par la suite de pouvoir obtenir d'autres mesures que celle qui est réalisée (notamment la matrice de confusion). Pour cela il est conseillé de couper le jeu de données en apprentissage (sur lequel sera appliqué le GridSearchCV) et un jeu de test avec 90/10.

In [ ]:
# Création du jeu d'apprentissage et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
from sklearn.model_selection import GridSearchCV


pipeline=Pipeline([("tfidf", TfidfVectorizer()),
                   ('svm', SVC())])

# creation des différents paramètres à tester pour SVM
# Attention dans le pipeline le nom pour le classifier SVM est : svm même si l'algorithme s'appelle SVC
# pour le référencer il faut utiliser le nom utilisé, i.e. svm, puis deux caractères soulignés
# et enfin le nom du paramètre
parameters = { 
     'svm__C': [0.001, 0.01, 0.1, 1, 10], 
    'svm__gamma' : [0.001, 0.01, 0.1, 1], 
    'svm__kernel': ['linear','rbf','poly','sigmoid']}
    

score='accuracy'

# Application de gridsearchcv, n_jobs=-1 permet de pouvoir utiliser plusieurs CPU s'ils sont disponibles
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1,scoring=score)

print("Application de gridsearch ...")
print("pipeline :", [name for name, _ in pipeline.steps])
print("parameters :")
print(parameters)
start_time = time.time()
grid_search.fit(X_train, y_train)
print("réalisé en  %0.3f s" % (time.time() - start_time))
print("Meilleur résultat : %0.3f" % grid_search.best_score_)

# autres mesures et matrice de confusion
y_pred = grid_search.predict(X_test)
MyshowAllScores(y_test,y_pred)


print("Ensemble des meilleurs paramètres :")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
# Affichage des premiers résultats du gridsearch
df_results=pd.concat([pd.DataFrame(grid_search.cv_results_["params"]),
           pd.DataFrame(grid_search.cv_results_["mean_test_score"], 
                        columns=[score])],axis=1).sort_values(score,ascending=False)
print ("\nLes premiers résultats : \n",df_results.head()) 


**Remarque :** dans GridSearchCV, le paramètre utilisé *n_jobs=-1* permet de pouvoir réaliser des traitements en parallèle en utilisant tous les CPU de la machine. 

Et maintenant pourquoi ne pas rechercher également les meilleurs paramètres aussi bien pour le pré-traitement des données que pour la vectorisation et le classifier.  

**Attention :** cette recherche peut bien entendu être très longue aussi dans le code suivant nous ne traitons que quelques paramètres et simplifions les hyperparamètres de SVM. 

Nous considérons à présent principalement les paramètres associés à TfidfVectorizer.

In [ ]:
pipeline=Pipeline([("cleaner", TextNormalizer()),
                   ("tfidf", TfidfVectorizer()),
                   ('svm', SVC())])


parameters = {
    'tfidf__stop_words':['english',None],
    'tfidf__lowercase': ['True','False'], 
     'svm__C': [1, 10], 
    'svm__gamma' : [1], 
    'svm__kernel': ['rbf']}
    
score='accuracy'
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1,  verbose=1, scoring=score)

print("Application de gridsearch ...")
print("pipeline :", [name for name, _ in pipeline.steps])
print("parameters :")
print(parameters)
start_time = time.time()
grid_search.fit(X_train, y_train)
print("réalisé en  %0.3f s" % (time.time() - start_time))
print("Meilleur résultat : %0.3f" % grid_search.best_score_)

# autres mesures et matrice de confusion
y_pred = grid_search.predict(X_test)
MyshowAllScores(y_test,y_pred)

print("Ensemble des meilleurs paramètres :")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
# Affichage des premiers résultats du gridsearch
df_results=pd.concat([pd.DataFrame(grid_search.cv_results_["params"]),
           pd.DataFrame(grid_search.cv_results_["mean_test_score"], 
                        columns=[score])],axis=1).sort_values(score,ascending=False)
print ("\nLes premiers résultats : \n",df_results.head())  



Enfin, nous examinons l'impact de quelques paramètres notamment la racinisation associées au prétraitement.

In [ ]:
pipeline=Pipeline([("cleaner", TextNormalizer()),
                   ("tfidf", TfidfVectorizer()),
                   ('svm', SVC())])


parameters = {
    'cleaner__getstemmer': ['True','False'],
    'cleaner__removedigit':['True','False'],
    'cleaner__removestopwords':['True','False'],
    'tfidf__lowercase': ['True','False'], 
    'svm__C': [1, 10], 
    'svm__gamma' : [1], 
    'svm__kernel': ['rbf']}
    
score='accuracy'
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1,  verbose=1,scoring=score)

print("Application de gridsearch ...")
print("pipeline :", [name for name, _ in pipeline.steps])
print("parameters :")
print(parameters)
start_time = time.time()
grid_search.fit(X_train, y_train)
print("réalisé en  %0.3f s" % (time.time() - start_time))
print("Meilleur résultat : %0.3f" % grid_search.best_score_)

# autres mesures et matrice de confusion
y_pred = grid_search.predict(X_test)
MyshowAllScores(y_test,y_pred)


print("Ensemble des meilleurs paramètres :")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
# Affichage des premiers résultats du gridsearch
df_results=pd.concat([pd.DataFrame(grid_search.cv_results_["params"]),
           pd.DataFrame(grid_search.cv_results_["mean_test_score"], 
                        columns=[score])],axis=1).sort_values(score,ascending=False)
print ("\nLes premiers résultats : \n",df_results.head()) 



La prochaine cellule s'intéresse à la lemmatisation et à son impact sur la classification.

In [ ]:
pipeline=Pipeline([("cleaner", TextNormalizer()),
                   ("tfidf", TfidfVectorizer()),
                   ('svm', SVC())])


parameters = {
    'cleaner__getlemmatisation': ['True','False'],
    'cleaner__removedigit':['True','False'],
    'cleaner__removestopwords':['True','False'],
    'tfidf__lowercase': ['True','False'], 
    'svm__C': [1, 10], 
    'svm__gamma' : [1], 
    'svm__kernel': ['rbf']}
    
score='accuracy'
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1,  verbose=1,scoring=score)

print("Application de gridsearch ...")
print("pipeline :", [name for name, _ in pipeline.steps])
print("parameters :")
print(parameters)
start_time = time.time()
grid_search.fit(X_train, y_train)
print("réalisé en  %0.3f s" % (time.time() - start_time))
print("Meilleur résultat : %0.3f" % grid_search.best_score_)

# autres mesures et matrice de confusion
y_pred = grid_search.predict(X_test)
MyshowAllScores(y_test,y_pred)


print("Ensemble des meilleurs paramètres :")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
# Affichage des premiers résultats du gridsearch
df_results=pd.concat([pd.DataFrame(grid_search.cv_results_["params"]),
           pd.DataFrame(grid_search.cv_results_["mean_test_score"], 
                        columns=[score])],axis=1).sort_values(score,ascending=False)
print ("\nLes premiers résultats : \n",df_results.head()) 



<font color=red>Exercice :</font> Comparer attentivement les  dernières exécutions. Que pouvez vous en déduire sur la meilleure configuration ? Quel est l'impact des paramètres de pré-traitement et de vectorisation ? Quel pouvez vous en déduire de manière générale ? 

<font color=blue>Solution :</font> 


In [ ]:
# Il s'avère que parfois les pré-traitements n'améliore pas forcément  (par exemple la lemmatisation)
# et que l'accuracy diminue.  Pourtant si l'on regarde la première expérimentation il s'avère qu'elle revient 
# à traiter le texte brut sans aucun pré-traitemet et que les quelques légers pré-traitement de base 
# suppression de caractères spéciaux, etc permettent d'améliorer très légérement les résultats (0.824 vs 0.825).  
# La question par contre, de manière générale, est comment améliorer ces résultats. En fait si l'on 
# regarde plus en détail les données on peut constater que le jeu de données n'est pas très volumineux et 
# qu'il n'y a pas forcément possibilité de mieux discriminer les avis positifs ou négatifs. 
# Pour améliorer la classification il existe cependant de nombreuses pistes d'autres pré-traitements 
# (e.g. ne retenir que des verbes, adjectifs souvent porteurs d'opinions et adverbe, 
# regarder plus spécifiquement les données positives et négatives (e.g. mieux analyser les données), 
# utiliser d'autres classifier ou d'autres types de classifiers (e.g. LR, CNN, etc.), utiliser 
# d'autres approches de représentation des données (e.g. des embeddings que nous verrons plus tard), 
# enrichir les données, ..... bref de très nombreuses pistes de recherche.

<font color=red>Exercice :</font> Dans le code ci-dessous, sélectionner un classifier et ses paramètres associés afin de tester s'il obtient de meilleurs résultats que SVM. Pour cela, il suffit de décommenter les lignes associèes au classifier que vous désirez et exécuter la cellule.  
Le code actuel permet de lancer la classifier LogisticRegression. Ne pas oublier de commenter les lignes associées pour tester d'autres classifiers.

In [ ]:
# Pour information le nombre de paramètres à tester a été défini de manière 
# à ce que les tests ne soient pas trop longs

pipeline=Pipeline([("cleaner", TextNormalizer()),
                   ("tfidf", TfidfVectorizer()),
    
                   # pour LogisticRegression enlever le commentaire de la ligne suivante
                   ("lr", LogisticRegression()),
    
                   # pour MultinomialNB enlever le commentaire de la ligne suivante 
                   #("mnb", MultinomialNB()),
    
                   # pour RandomForestClassifier enlever le commentaire de la ligne suivante
                   #('rfc', RandomForestClassifier())
                  ]
                 )


parameters = {
    # Pour logisticRegression enlever les commentaires des 3 lignes suivantes :
    'lr__solver' : ['newton-cg', 'lbfgs', 'liblinear'],
    'lr__penalty' : ['l2'],
    'lr__C' : [100, 10, 1.0, 0.1, 0.01],
    
    # Pour MulinomialNaiveBayes enlever les commentaires des 2 lignes suivantes :
    #'mnb__alpha': np.linspace(0.5, 1.5, 6),
    #'mnb__fit_prior': [True, False],  
    
    # pour RandomForestClassifier enlever les commentaires des 4 lignes suivantes :
    #'rfc__n_estimators': [500, 1200],
    #'rfc__max_depth': [25, 30],
    #'rfc__min_samples_split': [5, 10, 15],
    #'rfc__min_samples_leaf' : [1, 2], 
    }
     
    
score='accuracy'
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1,  verbose=1,scoring=score)

print("Application de gridsearch ...")
print("pipeline :", [name for name, _ in pipeline.steps])
print("parameters :")
print(parameters)
start_time = time.time()
grid_search.fit(X, y)
print("réalisé en  %0.3f s" % (time.time() - start_time))
print("Meilleur résultat : %0.3f" % grid_search.best_score_)

# matrice de confusion
y_pred = grid_search.predict(X_test)
MyshowAllScores(y_test,y_pred)

print("Ensemble des meilleurs paramètres :")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
# Affichage des premiers résultats du gridsearch
df_results=pd.concat([pd.DataFrame(grid_search.cv_results_["params"]),
           pd.DataFrame(grid_search.cv_results_["mean_test_score"], 
                        columns=[score])],axis=1).sort_values(score,ascending=False)
print ("\nLes premiers résultats : \n",df_results.head()) 




## **Sauvegarde du modèle**

Dans cette section nous sauvegardons le modèle pour pouvoir l'utiliser ultérieurement. Pour cela, nous utilisons simplement un pipeline en utilisant les paramètres et hyperparamètres qui ont été appris précédemment. Nous l'appliquons cette fois-ci à l'ensemble du jeu de données et non plus à un sous ensemble (X_train).  

In [ ]:

X=df.sentence
y=df.sentiment

pipeline=Pipeline([("cleaner", TextNormalizer()),
                   ("tfidf", TfidfVectorizer(lowercase='True')),
                   ('svm', SVC(C=1, gamma=1,kernel='rbf'))])
pipeline.fit(X,y)
filename='SentimentModel.pkl'
print("Sauvegarde du modèle dans ", filename)
pickle.dump(pipeline, open(filename, "wb"))

Maintenant qu'un modèle a été appris et sauvegardé .... il suffit de l'utiliser. Pour cela rendez vous sur le notebook **"4 - Utiliser un modele"** ...